<a href="https://colab.research.google.com/github/liuxiao916/Dive_into_DL_PyTorch/blob/main/3_3_%E7%BA%BF%E6%80%A7%E5%9B%9E%E5%BD%92%E7%9A%84%E7%AE%80%E6%B4%81%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.3 线性回归的简洁实现

In [1]:
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
import torch

## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(scale = 1,size=(num_examples,num_inputs)), dtype=torch.float)
labels = true_w[0]*features[:,0] + true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(scale = 0.01,size = labels.shape), dtype=torch.float)

## 3.3.2 读取数据集

In [3]:
import torch.utils.data as Data

batch_size = 10

dataset = Data.TensorDataset(features, labels)

data_iter = Data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True
    )

In [4]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[-1.4768,  1.2587],
        [ 0.9848, -1.5931],
        [-2.0431, -0.4522],
        [ 1.6726,  0.1236],
        [ 0.7577,  0.5423],
        [-0.4996, -1.9556],
        [ 0.9815, -0.6122],
        [ 0.5564,  0.3572],
        [-0.5396, -1.8347],
        [ 0.7831,  0.7508]]) tensor([-3.0507, 11.5744,  1.6576,  7.1348,  3.8566,  9.8510,  8.2269,  4.1029,
         9.3505,  3.2158])


## 3.3.3 定义模型

In [5]:
from torch import nn

net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


此处参考https://github.com/ShusenTang/Dive-into-DL-PyTorch/blob/master/docs/chapter03_DL-basics/3.3_linear-regression-pytorch.md

In [6]:
class LinearNet(nn.Module):
    def __init__(self, num_input):
        super(LinearNet, self).__init__()
        self.linear=nn.Linear(num_inputs, 1)
    def forward(self, x):
        y = self.linear(x)
        return y

Net = LinearNet(num_inputs)
print(Net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
for param in net.parameters():
    print(param)

print(net.state_dict())

Parameter containing:
tensor([[ 0.1588, -0.3627]], requires_grad=True)
Parameter containing:
tensor([0.7008], requires_grad=True)
OrderedDict([('linear.weight', tensor([[ 0.1588, -0.3627]])), ('linear.bias', tensor([0.7008]))])


## 3.3.4 初始化模型参数

In [8]:
#理论上pytorch已经默认初始化过了
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.normal_(net[0].bias, mean=0, std=0.01) 

Parameter containing:
tensor([-0.0003], requires_grad=True)

## 3.3.5 定义损失函数


In [9]:
criterion = nn.MSELoss()

## 3.3.6 定义优化算法


In [10]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.3.7 训练模型


In [11]:
num_epoch = 3
for i in range(1,num_epoch+1):
    for X,y in data_iter:
        output = net(X)
        loss = criterion(output,y.view(-1,1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch %d, loss: %f' % (i, loss.item()))

epoch 1, loss: 0.000373
epoch 2, loss: 0.000052
epoch 3, loss: 0.000145


In [12]:
dense = net[0]
true_w, dense.weight.data

([2, -3.4], tensor([[ 1.9996, -3.3997]]))

In [13]:
true_b, dense.bias.data

(4.2, tensor([4.1999]))